In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import warnings
from linearmodels.iv import IV2SLS
warnings.filterwarnings('ignore')
%matplotlib inline

# Local average treatment effects

In [2]:
data = pd.read_excel('hw4_data.xlsx')
df = data.copy()

In [3]:
df

,z,d1,d0,d,y1,y0,y
0,1,1,0,1,2.269907,0.484351,2.269907
1,0,0,0,0,3.549198,1.105060,1.105060
2,1,0,0,0,1.403377,0.814125,0.814125
3,0,1,0,0,1.648925,0.392743,0.392743
4,1,1,0,1,-0.354274,-0.752172,-0.354274
...,...,...,...,...,...,...,...
19995,1,0,0,0,1.307680,-0.340629,-0.340629
19996,1,0,0,0,4.273262,-0.225077,-0.225077
19997,1,0,0,0,3.735581,0.972001,0.972001
19998,1,1,0,1,-0.250311,-0.564228,-0.250311


## Generate the treatment effect variable and calculate the average treatment effect from the generated variable (i.e., the ATE for the whole population).

In [4]:
# Generate the treatment effect variable
df['treatment_effect_variable'] = df['y1'] - df['y0']
df['d1-d0'] = df['d1'] - df['d0']
df.head()

,z,d1,d0,d,y1,y0,y,treatment_effect_variable,d1-d0
0,1,1,0,1,2.269907,0.484351,2.269907,1.785556,1
1,0,0,0,0,3.549198,1.105060,1.105060,2.444138,0
2,1,0,0,0,1.403377,0.814125,0.814125,0.589252,0
3,0,1,0,0,1.648925,0.392743,0.392743,1.256181,1
4,1,1,0,1,-0.354274,-0.752172,-0.354274,0.397898,1


In [5]:
# average treatment effect for the whole population
df['treatment_effect_variable'].mean()

1.0060516975986729

In [6]:
df

,z,d1,d0,d,y1,y0,y,treatment_effect_variable,d1-d0
0,1,1,0,1,2.269907,0.484351,2.269907,1.785556,1
1,0,0,0,0,3.549198,1.105060,1.105060,2.444138,0
2,1,0,0,0,1.403377,0.814125,0.814125,0.589252,0
3,0,1,0,0,1.648925,0.392743,0.392743,1.256181,1
4,1,1,0,1,-0.354274,-0.752172,-0.354274,0.397898,1
...,...,...,...,...,...,...,...,...,...
19995,1,0,0,0,1.307680,-0.340629,-0.340629,1.648310,0
19996,1,0,0,0,4.273262,-0.225077,-0.225077,4.498339,0
19997,1,0,0,0,3.735581,0.972001,0.972001,2.763580,0
19998,1,1,0,1,-0.250311,-0.564228,-0.250311,0.313917,1


##  Generate the following four dummy variables

In [7]:
df["NT"] = 0 
df["DF"] = 0
df["COMP"] = 0
df["AT"] = 0 

df["NT"][(df['d1']==0) & (df['d0'] == 0)] = 1 # Never Taker
df["DF"][(df['d1']==0) & (df['d0'] == 1)] = 1 # Defier
df["COMP"][(df['d1']==1) & (df['d0'] == 0)] = 1 # Complier
df["AT"][(df['d1']==1) & (df['d0'] == 1)] = 1 # Always Taker

In [8]:
df

,z,d1,d0,d,y1,y0,y,treatment_effect_variable,d1-d0,NT,DF,COMP,AT
0,1,1,0,1,2.269907,0.484351,2.269907,1.785556,1,0,0,1,0
1,0,0,0,0,3.549198,1.105060,1.105060,2.444138,0,1,0,0,0
2,1,0,0,0,1.403377,0.814125,0.814125,0.589252,0,1,0,0,0
3,0,1,0,0,1.648925,0.392743,0.392743,1.256181,1,0,0,1,0
4,1,1,0,1,-0.354274,-0.752172,-0.354274,0.397898,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,0,0,0,1.307680,-0.340629,-0.340629,1.648310,0,1,0,0,0
19996,1,0,0,0,4.273262,-0.225077,-0.225077,4.498339,0,1,0,0,0
19997,1,0,0,0,3.735581,0.972001,0.972001,2.763580,0,1,0,0,0
19998,1,1,0,1,-0.250311,-0.564228,-0.250311,0.313917,1,0,0,1,0


## Report the proportions of always-takers, compliers, defiers, and never-takers.

In [9]:
never_takers_proportion = len(df[df["NT"] == 1])/len(df)
defiers_proportion = len(df[df["DF"] == 1])/len(df)
compliers_proportion = len(df[df["COMP"] == 1])/len(df)
alwaystakers_proportion = len(df[df["AT"] == 1])/len(df)
proportions = [never_takers_proportion, defiers_proportion, compliers_proportion, alwaystakers_proportion]
names = ['never_takers_proportion', 'defiers_proportion', 'compliers_proportion', 'alwaystakers_proportion']

In [10]:
for i in range(len(proportions)):
    print(f"{names[i]} = {proportions[i]}")

never_takers_proportion = 0.50055
defiers_proportion = 0.0
compliers_proportion = 0.3398
alwaystakers_proportion = 0.15965


## Report the ATE for always-takers

In [11]:
# average treatment effect for always_takers
df[df['AT'] == 1]['treatment_effect_variable'].mean()

-0.5051438462357073

## Report the ATE effect for never-takers.

In [12]:
# average treatment effect for never-takers
df[df['NT'] == 1]['treatment_effect_variable'].mean()

1.7963490987235717

## Report the ATE for compliers.

In [13]:
# average treatment effect for compliers
df[df['COMP'] == 1]['treatment_effect_variable'].mean()

0.5518992680521332

## Run OLS of y on d. Is the estimated coefficient on d similar to any ATE?
- ans : NO

In [14]:
model = ols('y ~ d', df).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.0648,0.011,5.845,0.000,0.043,0.087
d,-0.3524,0.018,-19.157,0.000,-0.388,-0.316


## Perform an IV regression of y on d by using z as an instrumental variable. Is the estimated coefficient on d similar to the ATE for compliers?
- ans : Yes


In [15]:
# first stage model(Z -> T)
stage1_model = ols('d ~ z', df).fit()
# reduced model(Z -> Y)
reduced_model = ols('y ~ z', data).fit()

In [16]:
# first stage model
stage1_model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.1570,0.005,31.188,0.000,0.147,0.167
z,0.3447,0.007,53.016,0.000,0.332,0.357


In [17]:
# reduced model
reduced_model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-0.1781,0.014,-12.651,0.000,-0.206,-0.150
z,0.1913,0.018,10.524,0.000,0.156,0.227


In [18]:
_2SLS = 0.1913 / 0.3447
_2SLS

0.5549753408761241

In [19]:
# average treatment effect for compliers
df[df['COMP'] == 1]['treatment_effect_variable'].mean()

0.5518992680521332

## Calculate the Wald estimator and verify that the Wald estimate is equivalent to the 2SLS estimator from the previous question. You can obtain the Wald estimate by computing the four conditional means.

In [20]:
E_Y_Z1 = df[df['z'] == 1]['y'].mean()
E_Y_Z0 = df[df['z'] == 0]['y'].mean()
E_D_Z1 = df[df['z'] == 1]['d'].mean()
E_D_Z0 = df[df['z'] == 0]['d'].mean()

Wald = (E_Y_Z1 - E_Y_Z0) / (E_D_Z1 - E_D_Z0)
Wald

0.5549258842110272

In [21]:
# 둘은 거의 같다고 볼 수 있다.
round(_2SLS,3) == round(Wald,3)

True